In [15]:
import pandas as pd
import numpy as np
df_policy = pd.read_csv('../Tbrain_Insurance/policy_claim/policy_0702.csv')
df_claim = pd.read_csv('../Tbrain_Insurance/policy_claim/claim_0702.csv')
df_training = pd.read_csv('../Tbrain_Insurance/training-set.csv')
df_testing = pd.read_csv('../Tbrain_Insurance/testing-set.csv')

## 0. Read Files

### policy

In [16]:
print(df_policy.shape) # 41 items  (1,747,942, 41)
print('unique:',df_policy['Policy_Number'].nunique()) #351,273 unique Policy Number
pd.set_option('display.max_columns', None)
df_policy.sample(10)   

(1747942, 41)
unique: 351273


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
692582,9fe9b3c83599ae0a7cbd25a537c988ad34f4e6af,Hd0d97640fb1d04b94d10434d17ef5b92b0086954,NaN,Y,f9862ca07accd6e45d42d2796e77abf735ac16d8,eccc856f0e611b57762984e758bb3cb5fee35abe,649cb76e2bd65391ec95a2769d4af25a6ca70564,2010,2000,10,70a39df6c3ab39ac618a952bc58818dcb1ab4e2c,5.0,P,車責,16G,3000000,0,30000000,0,5839,59.8,315b39a0fc19fb45d146c17b813d46c7acb0b872,0.0,10,1.20,-0.6,1,11/1965,2,1,584130e068c3f0f36bf0a7ef9308031af8fb6462,c1dfd96eea8cc2b62785275bca38ac261256e278,11/1965,0,0,0,0,0,0,0,
382443,fe66d6f78117f272db119906f0d7f14ea6ca7e3f,J8ebd3c8f722d446b005bdf627071474a0802b485,005b71724b69484c9df89c6e5e755b55607ddb2f,,75635d14089cb36873d703e70f97675a743b3c28,9e1816a8c2a63b7f96a82ce6487b2d43222750db,6bf0ae22034a9ebbc8379381b4051d32d00d5b33,2013,1798,10,9eb4f98b90af8bdb0cf379712394d1a1756fd7df,5.0,P,車責,18@,5000,50000,100000,0,82,68.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.30,-0.2,1,05/1962,1,1,e6de89674d9f1d7968a70dad4f41b844965b4fdc,761f22b2c1593d0bb87e0b606f990ba4974706de,09/1970,0,0,0,0,0,0,0,
468525,138b0988de191e61f538985540804bb7c6e9dfc3,Bfb52b29ded1212a3e187b0346ee83e88e9e419e9,NaN,Y,4f7d13e3a06436dcf2c47c9607ef8ad2402bb1a7,3a5e741326cb42b7283e6865856d6faae3543d2f,b61a62805c216f7eee9cb51e54a257beeff9bc0f,2015,1595,30,18267fa307a3edd872c66b0881cfef5986705339,5.0,P,車損,03L,0,0,1490000,0,367,149.0,39005d92752d56310b3a357f42b57f304621689a,0.0,1,-0.30,-0.2,1,04/1975,2,1,beba4d5d3ffb8fac7fe5ce87ac1eb2f75c4cd1a2,761f22b2c1593d0bb87e0b606f990ba4974706de,05/1974,0,0,0,0,0,0,0,
1283517,a4efe53bc765e523f59698a78daadb096a149e71,R4b59d4a9c0e99770dbcc0ce708a2bcc25b038713,0a169c1901697b7c3cf9f3dc738574b376629e9e,,07d7dcc8eec73de65123c774e5c51402a8d1f4d7,9e1816a8c2a63b7f96a82ce6487b2d43222750db,04c071f459a26dc52a19bf3eb92ff5eaec4acd01,2012,1500,10,9aba8deed17e700e5c9d49538f338f74e3d04ee4,5.0,P,車責,18@,5000,50000,100000,0,82,51.5,39005d92752d56310b3a357f42b57f304621689a,0.0,-1,-0.38,0.0,1,04/1967,2,1,86e55aec99bfd3868b399afe0c5069bd2a7b7d18,511a418e72591eb7e33f703f04c3fa16df6c90bd,01/1963,0,0,0,0,0,0,0,
1164633,2fc121af385039018dcc37a36f82221aa34a313f,A8fb9fb4befd3bf476eec7b0d3641b1e7c7609849,34d861e8d9f946371d0639752daef5923bc12365,,b880b14759bfecd440d9dc3036dce719e1b1b19f,960d139497003852038c328303d92f423e6854c4,d93186855f88ead75d402e5a9dfd583ad95457be,2008,1584,10,d7b9d65630a1116f0bbf1c7b293fcbd9ed69fc70,5.0,P,車責,16P,0,0,500000,0,1722,50.9,39005d92752d56310b3a357f42b57f304621689a,0.0,3,-0.10,0.0,1,12/1974,2,2,6e9b99384f21dc46400b5347a3eb341f6daeb334,761f22b2c1593d0bb87e0b606f990ba4974706de,12/1974,0,0,0,0,0,0,0,
777686,9d4a229e995c03dd5eca6d8a08eb85712f3bad6f,R3f83554ee391c6b9eb76ac1b211d1ca3886bfa61,fd1e0d5e17520f574bed179f05f92531128f3faf,,e89bc86c43c23a2fd8481a0519ad6d1b162c0ebe,9e1816a8c2a63b7f96a82ce6487b2d43222750db,bcaa72990e08089908de028708c4d2ad6a4321e1,2009,2362,10,4867cf98fcddd4e33762ca57f5cb547077ba9dcb,5.0,P,車損,20K,10000,0,50000,0,21,97.5,39005d92752d56310b3a357f42b57f304621689a,0.0,-1,-0.38,0.0,1,05/1956,2,1,29da9bc33b146db5b36af2908861e1de88c25fc8,511a418e72591eb7e33f703f04c3fa16df6c90bd,05/1956,0,0,0,0,0,0,0,
557187,c2f0aaf01c12bac1af5cf80702b7232869cc5e0a,270aab7fd489fcf7bb00e936dfaa0f66be35f09a1,52a8ce9e9938986fdc143be830f168a4554dd664,,2461bb562d2db758cfb6f9fb010049656b92a382,9e1816a8c2a63b7f96a82ce6487b2d43222750db,302b354c600a924a4f7ac7

In [17]:
'''Policy_Number	保單號碼 Insured's_ID	被保險人代號(替代值) Prior_Policy_Number	前保單號 Cancellation	非繼承自前保單 
Vehicle_identifier	車牌號碼 Vehicle_Make_and_Model1	廠牌名稱一 Vehicle_Make_and_Model2	廠牌名稱二
Manafactured_Year_and_Month	製造年份 Engine_Displacement_(Cubic_Centimeter)	排氣量 Imported_or_Domestic_Car	車系代號
Coding_of_Vehicle_Branding_&_Type	廠牌車型代號
qpt	乘載數量 fpt	乘載單位 Main_Insurance_Coverage_Group	險種分類 Insurance_Coverage	險種代碼
Insured_Amount1	保額一 Insured_Amount2	保額二 Insured_Amount3	保額三
Coverage_Deductible_if_applied	自負額 Premium	簽單保費 Replacement_cost_of_insured_vehicle	重置成本
Distribution_Channel	主通路代號 Multiple_Products_with_TmNewa_(Yes_or_No?)	同一車主前一年度非車險保單件數
lia_class	車責本期級數 plia_acc	車責賠款係數 pdmg_acc	車體賠款係數 fassured	被保險人性質
ibirth	出生日期 fsex	性別 fmarriage	婚姻狀況 aassured_zip	郵遞區號
iply_area	承保地區代號(分公司) dbirth	要保人出生日期 fequipment1	配備-音響註記 fequipment2	配備-車箱註記 
fequipment3	配備-冷凍機組註記 fequipment4	配備-車框、車斗註記 fequipment5	配備-昇降系統註記 fequipment6	配備-電動車電池註記
fequipment9	配備-其他 nequipment9	配備-其他說明
'''
print(df_policy.columns)

Index(['Policy_Number', 'Insured's_ID', 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3',
       'Coverage_Deductible_if_applied', 'Premium',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9'],
      dtype='object')


In [18]:
same_cols=["Insured's_ID", 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9']

diff_cols=['Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3','Coverage_Deductible_if_applied', 'Premium']

In [26]:
# Convert 'Main_Insurance_Coverage_Group', 'Insurance_Coverage' to values, 
# so we can create features from diff_cols afterwards

# get dummy on Main_Insurance_Coverage_Group & Insurance_Coverage
df = df_policy
segment=['Main_Insurance_Coverage_Group','Insurance_Coverage']
df_dummy = pd.get_dummies(df, columns=segment)
print(df_dummy.shape)
df_dummy.head()

(1747942, 102)


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [43]:
# replace diff_col with new columns name (after get dummy) 
#diff_col = df_dummy.columns[39:] # ++ ['Coverage_Deductible_if_applied', 'Premium']
diff_cols=['Main_Insurance_Coverage_Group_竊盜', 'Main_Insurance_Coverage_Group_車損',
       'Main_Insurance_Coverage_Group_車責', 'Insurance_Coverage_00I',
       'Insurance_Coverage_01A', 'Insurance_Coverage_01J',
       'Insurance_Coverage_02K', 'Insurance_Coverage_03L',
       'Insurance_Coverage_04M', 'Insurance_Coverage_05E',
       'Insurance_Coverage_05N', 'Insurance_Coverage_06F',
       'Insurance_Coverage_07P', 'Insurance_Coverage_08H',
       'Insurance_Coverage_09@', 'Insurance_Coverage_09I',
       'Insurance_Coverage_10A', 'Insurance_Coverage_12L',
       'Insurance_Coverage_14E', 'Insurance_Coverage_14N',
       'Insurance_Coverage_15F', 'Insurance_Coverage_15O',
       'Insurance_Coverage_16G', 'Insurance_Coverage_16P',
       'Insurance_Coverage_18@', 'Insurance_Coverage_18I',
       'Insurance_Coverage_20B', 'Insurance_Coverage_20K',
       'Insurance_Coverage_25G', 'Insurance_Coverage_26H',
       'Insurance_Coverage_27I', 'Insurance_Coverage_29B',
       'Insurance_Coverage_29K', 'Insurance_Coverage_32N',
       'Insurance_Coverage_33F', 'Insurance_Coverage_33O',
       'Insurance_Coverage_34P', 'Insurance_Coverage_35H',
       'Insurance_Coverage_36I', 'Insurance_Coverage_37J',
       'Insurance_Coverage_40M', 'Insurance_Coverage_41E',
       'Insurance_Coverage_41N', 'Insurance_Coverage_42F',
       'Insurance_Coverage_45@', 'Insurance_Coverage_46A',
       'Insurance_Coverage_47B', 'Insurance_Coverage_51O',
       'Insurance_Coverage_55J', 'Insurance_Coverage_56B',
       'Insurance_Coverage_56K', 'Insurance_Coverage_57C',
       'Insurance_Coverage_57L', 'Insurance_Coverage_65K',
       'Insurance_Coverage_66C', 'Insurance_Coverage_66L',
       'Insurance_Coverage_67D', 'Insurance_Coverage_68E',
       'Insurance_Coverage_68N', 'Insurance_Coverage_70G',
       'Insurance_Coverage_70P', 'Insurance_Coverage_71H',
       'Insurance_Coverage_72@','Coverage_Deductible_if_applied', 'Premium']

In [48]:
# group by same cols, and Find Policy Number that has the most records
df_policy_grouped = df_dummy.groupby(['Policy_Number'])
df_policy_grouped_count = df_dummy.groupby(['Policy_Number'])[diff_cols].count()
print(np.max(df_policy_grouped_count['Main_Insurance_Coverage_Group_竊盜']))

18


In [23]:
## Examine data: find same and diff cols
# df_policy_grouped_count[df_policy_grouped_count['Main_Insurance_Coverage_Group_竊盜']==18]
# # check 自然人  policy
# df_policy[df_policy['Policy_Number']=='1ba69ae26d09693bd4ce0ae7deb373bdf427dafd']
# df_policy[df_policy['Policy_Number']=='81208dfe578ee4bc89774c16d24799924e78f0e0'].nunique()
# # check 法人  policy
# df_policy[df_policy['Policy_Number']=='17ccb5ec55e642270298b3bbc959ff7c0ecfa6b1']

In [47]:
print('unique:',df_policy['Policy_Number'].nunique()) 
print(df_policy_grouped_count.shape)  # == 351,273 unique Policy Number
df_policy_grouped_count.head()

unique: 351273
(351273, 65)


,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@,Coverage_Deductible_if_applied,Premium
Policy_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000061141b237e8619efedcd6939fddeff05b9a5,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
000148bbf40f28bfc860c0d272aadc429e19e2d1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


## Create Features from diff_cols

In [51]:
# Main_Insurance_Coverage_Group	險種分類 Insurance_Coverage	險種代碼 Insured_Amount1	保額一 Insured_Amount2	保額二 
# Insured_Amount3	保額三 Coverage_Deductible_if_applied	自負額 Premium	簽單保費

In [52]:
df_policy_grouped_sum = df_dummy.groupby(['Policy_Number'])[diff_cols].sum()

In [57]:
# df_policy_grouped_sum.info()

In [58]:
print('unique:',df_policy['Policy_Number'].nunique()) 
print(df_policy_grouped_sum.shape) 
df_policy_grouped_sum.head()

unique: 351273
(351273, 65)


,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@,Coverage_Deductible_if_applied,Premium
Policy_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000061141b237e8619efedcd6939fddeff05b9a5,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1586
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,2,2,3,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,9132
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,2861
000148bbf40f28bfc860c0d272aadc429e19e2d1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1728
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2195


## features from dates

In [59]:
### examine Drivers
driver = df_policy[['ibirth' , 'dbirth']]
print(driver.shape[0])
driver.head()

1747942


,ibirth,dbirth
0,08/1984,08/1984
1,08/1984,08/1984
2,10/1975,10/1975
3,10/1975,10/1975
4,12/1989,12/1989


In [60]:
condition1 = driver['ibirth'].isnull()    # 18.2 %
condition2 = driver['dbirth'].isnull()    # 14.8 %

condition3 = driver['ibirth'].notnull()   
condition4 = driver['dbirth'].notnull()   


driver_0N= driver[condition3&condition4]  # both have values  (81.5%)
driver_1N= driver[condition1^condition2]  # missing one (3.9%)
driver_2N= driver[condition1|condition2]  # missing both (18.4%)

print(driver_0N.shape[0]/driver.shape[0])
print(driver_1N.shape[0]/driver.shape[0]) 
print(driver_2N.shape[0]/driver.shape[0])

0.8152427254451234
0.039585409584528546
0.18475727455487653


In [61]:
# check indiff. in 0N

indiff = driver_0N[driver_0N['ibirth']!=driver_0N['dbirth']]
print(indiff.shape[0])
indiff.sample(10)

75794


,ibirth,dbirth
16717,01/1977,07/1979
563942,03/1962,11/1981
169095,09/1951,12/1976
350421,06/1972,08/1972
955515,03/1953,05/1951
313060,07/1947,11/1978
411615,01/1955,11/1979
37312,06/1983,11/1982
166010,12/1957,08/1982
1190007,01/1979,05/1969


In [62]:
### dbirth has some weird date
### check non-null data & sort
print(driver['dbirth'].notnull().sum())
import datetime
a = df_policy[driver['dbirth'].notnull()]['dbirth']
b = sorted(a, key=lambda x: datetime.datetime.strptime(x, '%m/%Y'), reverse = True)

1489010


In [63]:
replace_date = {
    '09/2658',
 '09/2658',
 '09/2658',
 '09/2658',
 '10/2622',
 '10/2622',
 '10/2622',
 '10/2622',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2581',
 '10/2581',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '10/2521',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '10/2472',
 '10/2472',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '11/2464',
 '11/2464',
 '11/2464',
 '03/2464',
 '03/2464',
 '12/2453',
 '12/2453',
 '12/2453',
 '12/2453',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152'}
len(replace_date)

19

In [64]:
replace_date_set = set(replace_date)
len(replace_date_set)

19

In [65]:
date_switch = df_policy.apply(lambda row: True if row['dbirth'] in replace_date_set else False, axis = 1)
df_policy_selected = df_policy[date_switch]
print(df_policy_selected.shape[0])
df_policy_selected.head()

92


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
23221,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,竊盜,05N,0,0,1500000,10,2818,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23222,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,竊盜,09@,0,0,1500000,0,423,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23223,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,16G,3000000,0,36000000,0,2178,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23224,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,16P,0,0,600000,0,1680,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23225,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車損,20K,10000,0,50000,0,21,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,


In [66]:
selected_idx = df_policy_selected.index.values
selected_idx

array([  23221,   23222,   23223,   23224,   23225,  162161,  162162,
        234348,  234349,  277667,  277668,  277669,  277670,  277671,
        277672,  283765,  283766,  283767,  283768,  283769,  283770,
        283771,  283772,  324704,  324705,  324706,  324707,  324708,
        324709,  324710,  324711,  324712,  418357,  418358,  418359,
        418360,  541847,  665698,  665699,  665700,  665701,  896783,
        896784,  896785,  896786,  896787,  896788,  896789,  896790,
        896791,  972345,  972346,  972347,  972348,  999213,  999214,
       1106167, 1106168, 1106169, 1106170, 1106171, 1420273, 1420274,
       1420275, 1420276, 1420277, 1457082, 1457083, 1457084, 1457085,
       1461117, 1461118, 1461119, 1521091, 1521092, 1521093, 1521094,
       1521095, 1521096, 1608188, 1608189, 1608190, 1608191, 1608192,
       1745912, 1745913, 1745914, 1745915, 1745916, 1745917, 1745918,
       1745919])

In [67]:
df_policy_selected_date = df_policy[['ibirth', 'dbirth','fassured']].loc[df_policy_selected.index]
# "1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"
df1 = df_policy_selected_date[df_policy_selected_date['fassured']==1]
df2 = df_policy_selected_date[df_policy_selected_date['fassured']==2]
print(df1.shape[0])
print(df2.shape[0])

60
32


In [68]:
df2.head() ## replace as NaN  (as ibirth)

,ibirth,dbirth,fassured
23221,NaN,08/2569,2
23222,NaN,08/2569,2
23223,NaN,08/2569,2
23224,NaN,08/2569,2
23225,NaN,08/2569,2


In [69]:
df1.head() ## replace as ibirth

,ibirth,dbirth,fassured
324704,04/1979,04/2579,1
324705,04/1979,04/2579,1
324706,04/1979,04/2579,1
324707,04/1979,04/2579,1
324708,04/1979,04/2579,1


In [70]:
df_policy['ID']=df_policy.index   # create ID column for ref.
df_policy.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9,ID
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,1
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,2
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16P,0,0,500000,0,1098,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,3
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,f4c3b98bf06237d20aa5f6c73b0c5359019e1cf8,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0,1,12/1989,2,2,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,,4


In [71]:
### replace weird date as ibirth
df_policy['dbirth_new'] = df_policy.apply(lambda row: row['ibirth'] if row['ID'] in selected_idx 
                                          else row['dbirth'], axis = 1)

In [72]:
df_policy[['ibirth', 'dbirth', 'fassured','dbirth_new']].loc[selected_idx].sample(10)

,ibirth,dbirth,fassured,dbirth_new
283770,NaN,07/2280,2,NaN
324704,04/1979,04/2579,1,04/1979
1420276,02/1951,01/2512,1,02/1951
1106171,NaN,11/2512,2,NaN
1608188,08/1967,08/2471,1,08/1967
665699,10/1982,10/2622,1,10/1982
896786,02/1952,02/2152,1,02/1952
1745915,11/1980,11/2602,1,11/1980
972345,09/1985,09/2658,1,09/1985
665700,10/1982,10/2622,1,10/1982


In [73]:
df_policy['dbirth'] = df_policy['dbirth_new']
df_policy = df_policy.drop(['dbirth_new','ID'], axis =1)
df_policy.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16P,0,0,500000,0,1098,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,f4c3b98bf06237d20aa5f6c73b0c5359019e1cf8,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0,1,12/1989,2,2,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,


In [74]:
print(df_policy.shape)

(1747942, 41)


In [ ]:
create features from these datetime columns later....

### Claim

In [ ]:
print(df_claim.shape)  # 20 items
print('unique:',df_claim['Policy_Number'].nunique()) #35895 unique Policy Number
pd.set_option('display.max_columns', None)  ## show all columns
df_claim.head()

In [ ]:
print(df_claim.columns)

In [ ]:
df_claim.nunique()              
# shape(69613, 20)
# unique Policy_Number: 35895 

In [ ]:
df_claim_grouped = df_claim.groupby(['Policy_Number','Claim_Number'])
df_claim_grouped_count = df_claim.groupby(['Policy_Number','Claim_Number']).count()
df_claim_grouped_count[df_claim_grouped_count['Nature_of_the_claim']==12].head()

In [ ]:
df_claim[df_claim['Policy_Number']=='4cd5fd8a86f0f5cd10fa8b416eba56582d20b573']
# Nature_of_the_claim	1.賠付 2.追償
# Salvage_or_Subrogation? 追償金額 (Nature_of_the_claim 才有值)
# Driver's_Replationship_with_Insured "1:被保險人本人 2:被保險人親友 3:被保險人員工 4:租用被保險車輛 5:其他, 6:配偶, 7:子女"

### training & testing

In [ ]:
print(df_training.shape)
print('unique:',df_training['Policy_Number'].nunique()) #210763 unique Policy Number
df_training.head()

In [ ]:
df_training[df_training['Next_Premium']==100]  # if Next_Premium = 100 -> testing_set

In [ ]:
print(df_testing.shape)
print('unique:',df_testing['Policy_Number'].nunique()) #140510 unique Policy Number
df_testing.head()

In [ ]:
a=df_training['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()*100
b=df_testing['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()*100
print('training set (pct)  %f '%a)
print('testing set (pct)  %f '%b)

## 1. Combine policy & claim & training & testing

In [ ]:
df_all= df_policy.merge(df_claim, on=df_policy.columns[0], how='left', suffixes=('','_Claim'))
print(df_all.columns)
print(df_all.shape)   # 60 items (overlap on Policy_Number only)

In [ ]:
df_all= df_all.merge(df_training, on=df_policy.columns[0], how='left')
df_all= df_all.merge(df_testing, on=df_policy.columns[0], how='left')
df_all['Next_Premium'] = df_all.apply(
    lambda row: row['Next_Premium_y'] if row['Next_Premium_y']==100 else row['Next_Premium_x'], axis = 1)
df_all=df_all.drop(['Next_Premium_x','Next_Premium_y'], axis =1)
print(df_all.shape) 
print(df_all.columns)

In [ ]:
df_all.info()

### 1.1 check missing data

In [ ]:
# % percentage
df_all.isnull().sum()/df_all.shape[0]*100   # about 22.7 declaire claim

In [ ]:
### df_all.nunique()

# '''
# Policy_Number                                 351273  Encode
# Insured's_ID                                  299999  Encode
# Prior_Policy_Number                           281678  Encode
# Cancellation                                       2  Dummy
# Vehicle_identifier                            347882  Encode
# Vehicle_Make_and_Model1                          136  Encode (too many...)
# Vehicle_Make_and_Model2                         8112  Encode
# Manafactured_Year_and_Month                       49  Datetime
# Engine_Displacement_(Cubic_Centimeter)           768  no Change
# Imported_or_Domestic_Car                          10  Dummy
# Coding_of_Vehicle_Branding_&_Type               7405  Encode
# qpt                                               19  Dummy
# fpt                                                1  Dummy
# Main_Insurance_Coverage_Group                      3  Dummy
# Insurance_Coverage                                60  Dummy
# Insured_Amount1                                  111  no Change
# Insured_Amount2                                   89  no Change
# Insured_Amount3                                 3071  no Change
# Coverage_Deductible_if_applied                    53  no Change
# Premium                                        18238  no Change
# Replacement_cost_of_insured_vehicle             2947  no Change
# Distribution_Channel                             875  Encode (too many...)
# Multiple_Products_with_TmNewa_(Yes_or_No?)       179  no Change (dummy??)
# lia_class                                         21  no Change (dummy??)
# plia_acc                                          21  no Change (dummy??)
# pdmg_acc                                          13  no Change (dummy??)
# fassured                                           4  Dummy
# ibirth                                           918  Datetime
# fsex                                               4  Dummy
# fmarriage                                          4  Dummy 
# aassured_zip                                    1722  Encode (too many...)                                                                       ...  
# iply_area                                         22  Dummy
# dbirth                                           945  Datetime
# fequipment1                                        2  Dummy
# fequipment2                                        2  Dummy
# fequipment3                                        2  Dummy
# fequipment4                                        1  Dummy
# fequipment5                                        2  Dummy
# fequipment6                                        1  Dummy
# fequipment9                                        2  Dummy 
# nequipment9                                        6  Dummy
# ============ Claim  ========
# Claim_Number                                   41137  Encode
# Nature_of_the_claim                                2  Dummy
# Driver's_Gender                                    2  Dummy
# Driver's_Relationship_with_Insured                 7  Dummy
# DOB_of_Driver                                    799  Datetime
# Marital_Status_of_Driver                           2  Dummy
# Accident_Date                                     25  Datetime
# Cause_of_Loss                                     17  Dummy
# Paid_Loss_Amount                               17229  no Change 
# paid_Expenses_Amount                             799  no Change
# Salvage_or_Subrogation?                         2056  no Change
# Coverage                                          48  Encode (same as Insurance_Coverage!)
# Vehicle_identifier_Claim                       35714  Encode (same as Vehicle_identifier if applicable!)
# At_Fault?                                         23  no Change (dummy??)
# Claim_Status_(close,_open,_reopen_etc)             2  Dummy
# Deductible                                        67  no Change 
# Accident_area                                     22  Dummy  (same as aassured_zip?)
# number_of_claimants                               14  no Change (dummy??)
# Accident_Time                                     39  DateTime
# Next_Premium                                   22817  no Change
# '''

In [ ]:
## fill na as 0 (Check explaination document, those 0 doesn't mean anything)

df= df_all
cols = ['Prior_Policy_Number','Vehicle_identifier','fsex','fmarriage','Claim_Number','Nature_of_the_claim','Policy_Number',
        "Driver's_Gender","Driver's_Relationship_with_Insured",'Marital_Status_of_Driver',
        'Cause_of_Loss','number_of_claimants']
for col in cols:
    df[col]=df[col].fillna(0)
    print(col, df[col].isnull().sum())


In [ ]:
df_all = df

In [ ]:
## fill na as -1 ( becasue 0 means something else)

df= df_all
cols = ['Paid_Loss_Amount','paid_Expenses_Amount','Salvage_or_Subrogation?', 'At_Fault?', 
        'Claim_Status_(close,_open,_reopen_etc)', 'Deductible']
for col in cols:
    df[col]=df[col].fillna(-1)
    print(col, df[col].isnull().sum())



In [ ]:
df_all = df

In [ ]:
##### Verify and drop 
df = df_all
### verify if 'Vehicle_identifier' = 'Vehicle_identifier_Claim'  (YES)
df_temp=df[['Policy_Number','Vehicle_identifier','Vehicle_identifier_Claim']]
df_temp2 = df_temp[df_temp['Vehicle_identifier_Claim'].notnull()] 
print(df_temp2.shape)
df_temp2[df_temp2['Vehicle_identifier']==df_temp2['Vehicle_identifier_Claim']]
print(df_temp2.shape)     

### verify if 'Insurance_Coverage'=='Coverage'    (YES)
df_temp=df[['Policy_Number','Insurance_Coverage','Coverage']]
df_temp2 = df_temp[df_temp['Coverage'].notnull()] 
print(df_temp2.shape)
df_temp2.head()
df_temp2[df_temp2['Insurance_Coverage']==df_temp2['Coverage']]
print(df_temp2.shape)  # 'Insurance_Coverage'=='Coverage' if there is value

### verify if 'Accident_area'=='iply_area'      (YES)
df_temp=df[['Policy_Number','Accident_area','iply_area']]
df_temp2 = df_temp[df_temp['Accident_area'].notnull()] 
print(df_temp2.shape)
df_temp2.head()
df_temp2[df_temp2['iply_area']==df_temp2['Accident_area']]
print(df_temp2.shape)  # 'Accident_area'=='iply_area'' if there is value

In [ ]:
## drop 'Coverage' & 'Vehicle_identifier_Claim' & 'Accident_area
df= df_all
df = df.drop(['Coverage','Vehicle_identifier_Claim','Accident_area'], axis=1)
df.shape

In [ ]:
df_all = df

In [ ]:
### fill NaN as (today)

# df= df_all
# cols = ['ibirth','dbirth','DOB_of_Driver','Accident_Date','Accident_Time']
# for col in cols:
#     df[col]=df[col].fillna(0)
#     print(col, df[col].isnull().sum())

In [ ]:
df_all = df

In [ ]:
# df_all.isnull().sum()   

In [ ]:
df = df_all
df[df.columns[41:]].nunique()

### 1.2 Label Encoder: convert anon id to No.

In [ ]:
pd.set_option('display.max_columns', None)  ## show all columns
print(df_all.shape)
df_all.head()

In [ ]:
'''
encoded columns:

Policy_Number	保單號碼
Insured's_ID	被保險人代號(替代值)
Prior_Policy_Number	前保單號
Vehicle_identifier	車牌號碼                               ## same as vehicle_identifier
Vehicle_Make_and_Model1	廠牌名稱一                   
Vehicle_Make_and_Model2	廠牌名稱二
Coding_of_Vehicle_Branding_&_Type	廠牌車型代號
Distribution_Channel	主通路代號
aassured_zip	郵遞區號                            
Claim_Number	賠案號碼
Vehicle_identifier_Claim	車牌                           ## same as vehicle_identifier (already drop)

'''

In [ ]:
df = df_all

## do not include:  Vehicle_identifier_Claim	車牌  (map to Vehicle_identifier)
cols = ('Policy_Number',"Insured's_ID",'Prior_Policy_Number','Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2','Coding_of_Vehicle_Branding_&_Type','Distribution_Channel','aassured_zip',
        'Distribution_Channel','aassured_zip','Claim_Number')


# process columns, apply LabelEncoder to categorical features
from sklearn.preprocessing import LabelEncoder
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))

# shape        
print('Shape of data: {}'.format(df.shape))
df.head()

In [ ]:
### Already dropped
### encode 'Vehicle_identifier_Claim' (NaN: NaN, not NaN: 'Vehicle_identifier' )
# df = df_all
# condition1 = df['Vehicle_identifier_Claim'].isnull()
# condition2 = df['Vehicle_identifier_Claim'].notnull()

# df['Vehicle_identifier_Claim'][condition1]=df['Vehicle_identifier_Claim']
# df['Vehicle_identifier_Claim'][condition2]=df['Vehicle_identifier']

#df[['Policy_Number','Vehicle_identifier','Vehicle_identifier_Claim']].sample(5)

### 1.3 Get dummy

In [ ]:

'''
dummy columns:

Cancellation                   非繼承自前保單	類別	Y: 是 、空白: 否(N)
Imported_or_Domestic_Car       車系代號	類別	參照：車系代號Sheet
qpt                            乘載數量	數值
fpt                            乘載單位	類別	P:載客 T:噸
Main_Insurance_Coverage_Group  險種分類	類別	參照:險種分類及自負額說明Sheet
Insurance_Coverage    ####     險種代碼	類別	參照:險種分類及自負額說明Sheet
fassured                       被保險人性質	類別	"1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"
fsex                           性別	類別	1:男 2:女 空白:法人(0)
fmarriage                      婚姻狀況	類別	1:已婚 2:未婚
iply_area             ####     承保地區代號(分公司)	類別
fequipment1                    配備-音響註記	類別	1:是 0:否
fequipment2                    配備-車箱註記	類別	1:是 0:否
fequipment3                    配備-冷凍機組註記	類別	1:是 0:否
fequipment4                    配備-車框、車斗註記	類別	1:是 0:否
fequipment5                    配備-昇降系統註記	類別	1:是 0:否
fequipment6                    配備-電動車電池註記	類別	1:是 0:否
fequipment9                    配備-其他	類別	1:是 0:否
nequipment9                    配備-其他說明	類別	1:是 0:否
Nature_of_the_claim            賠案性質	類別	1.賠付 2.追償
Driver's_Gender                肇事駕駛性別	類別	1:男 2:女 空白:法人
Driver's_Relationship_with_Insured 與被保險人關係	類別	"1:被保險人本人 2:被保險人親友 3:被保險人員工 4:租用被保險車輛 5:其他, 6:配偶, 7:子女"
Marital_Status_of_Driver       肇事駕駛婚姻	類別	1:已婚 2:未婚
Cause_of_Loss                  出險原因	類別
Coverage             ###       險種代號	類別  same as insurance_coverage (Already Dropped)
Accident_area        ####      出險地區	類別  same as iply (AlreadyDrop)
'''

In [ ]:
## Label cancellation from {'Y', ' '}-> {'Y', 'N'} get dummy later
df= df_all
print(set(df['Cancellation']))
input1 = ' '
replace1 = 'N'
df['Cancellation']=df['Cancellation'].replace(input1, replace1) 
df.head()

In [ ]:
df_all = df

In [ ]:
### Getting dummy categorical features:

In [ ]:
df = df_all
segment=['Cancellation','Imported_or_Domestic_Car','qpt','fpt','Main_Insurance_Coverage_Group','Insurance_Coverage',
         'fassured', 'fsex','fmarriage','iply_area', 'fequipment1','fequipment2','fequipment3','fequipment4',
         'fequipment5','fequipment6','fequipment9','nequipment9','Nature_of_the_claim',"Driver's_Gender",
         "Driver's_Relationship_with_Insured", 'Marital_Status_of_Driver', 'Cause_of_Loss']
df_dummy = pd.get_dummies(df, columns=segment)
print(df_dummy.shape)
df_dummy.head()

### 1.4 convert object (date) to datetime format

In [ ]:
## check date format
date_cols=['ibirth','dbirth','DOB_of_Driver','Accident_Date','Accident_Time']


In [ ]:
df_dummy[date_cols].info()
"""
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Data columns (total 5 columns):
ibirth           object
dbirth           object               # object... (selected index repalce with ibirth)
DOB_of_Driver    object
Accident_Date    object
Accident_Time    object
dtypes: datetime64[ns](4), object(1)
memory usage: 91.0+ MB
"""

In [ ]:
df_dummy[date_cols].sample(10)  ## 77% missing data

In [ ]:
type(df_dummy['Accident_Time'][1000000])

In [ ]:
### fill NaN as (2020)
df= df_dummy

cols = ['Accident_Time']
for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna(str('00:00'))
    print(col, df[col].isnull().sum())

In [ ]:
type(df['Accident_Time'][1])
df['Accident_Time'].sample(10)

In [ ]:
set(df['Accident_Time'])

In [ ]:
### fill NaN as (2020)

cols = ['ibirth','dbirth','DOB_of_Driver']

for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna('09/2018')
    print(col, df[col].isnull().sum())

cols = ['Accident_Date']
for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna('2018/09')
    print(col, df[col].isnull().sum())

df.isnull().sum().sum()    
df_dummy= df

In [ ]:
df_dummy[date_cols].sample(10)

In [ ]:
df= df_dummy

date_col={'ibirth','dbirth','DOB_of_Driver'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%m/%Y', errors='ignore')  #'%Y-%m-%d %H:%M:%S'   #errors='coerce'

date_col={'Accident_Date'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%Y/%m',errors='ignore')  #'%Y-%m-%d %H:%M:%S'

date_col={'Accident_Time'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%H:%M',errors='ignore') #'%Y-%m-%d %H:%M:%S'

df_dummy_date = df

In [ ]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].info(verbose= True)

### 1.4.1 Extract Features from datetime data then drop datetime64 columns

In [ ]:
Claim_year = 2016
yearsdiff = Claim_year-df_dummy_date['Manafactured_Year_and_Month']
df_dummy_date['V_age']=yearsdiff
df_dummy_date['V_age'].head()

In [ ]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].loc[932405]

In [ ]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].sample(10)

In [ ]:
## Today
today = datetime.date.today()
daysdiff = today-df_dummy_date['ibirth']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()

In [ ]:
years = components.days/365
df_dummy_date['i_age'] = years
df_dummy_date.head()

In [ ]:
today = datetime.date.today()
daysdiff = today-df_dummy_date['dbirth']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
years = components.days/365
df_dummy_date['d_age'] = years
df_dummy_date['d_age'].head()

In [ ]:
today = datetime.date.today()
daysdiff = today-df_dummy_date['DOB_of_Driver']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()
years = components.days/365
df_dummy_date['Dr_age'] = years
df_dummy_date['Dr_age'].head()

In [ ]:
# set(df_claim['Accident_Date'])  # between 2015/01 -> 2016/12    24months

In [ ]:
daysdiff = today-df_dummy_date['Accident_Date']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
months = components.days/30
df_dummy_date['Accident_Month_ago'] = months
df_dummy_date['Accident_Month_ago'].sample(10)

In [ ]:
from datetime import datetime, date
df_dummy_date['Acc_DayOfWeek'] = df_dummy_date.apply(lambda row: row['Accident_Date'].weekday(), axis =1)
df_dummy_date['Acc_DayOfWeek'][:5]  

In [ ]:
daysdiff = today-df_dummy_date['Accident_Time']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
hours = components.hours
df_dummy_date['Accident_Hour'] = hours
df_dummy_date['Accident_Hour'].sample(10)

In [ ]:
df_dummy_date.head()

In [ ]:
df_dummy_date[date_cols].head()

In [ ]:
drop_date_cols = ['ibirth', 'dbirth', 'DOB_of_Driver','Accident_Date', 'Accident_Time']
df_dummy_date = df_dummy_date.drop (drop_date_cols, axis = 1)

In [ ]:
print(df_dummy_date.shape)
df_dummy_date.head()

In [ ]:
df_dummy_date.info()

### 1.5 convert object (string) to numeric

In [ ]:
# df[] = pd.to_numeric(df[], errors='coerce')

## 2. Map features to unique Policy Number

## 3. Define Train, Test, Y

In [ ]:
#df_dummy_date.isnull().sum()

In [ ]:
df = df_dummy_date 

df_train = df[df['Next_Premium']!=100]
print(df_train.shape)
df_test = df[df['Next_Premium']==100]
print(df_test.shape)
print(df_train.shape[0]+df_test.shape[0])
print(df.shape[0])

In [ ]:
df.isnull().sum().sum()

In [ ]:
y=df_train['Next_Premium']
y.head()

In [ ]:
df_train=df_train.drop(['Next_Premium'], axis=1)
print(df_train.shape)

In [ ]:
df_test=df_test.drop(['Next_Premium'], axis=1)
print(df_test.shape)

### Modelling

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
train = df_train
test = df_test
y_train = y

In [ ]:
# ## downsampling for testing:
# train = df_train.head(50000) # 1,212,659
# y_train = y.head(50000)
# ## downsampling for testing:
# test=df_test.head(3000)   #776,268

In [ ]:
# train.to_csv('train.csv')
# test.to_csv('test.csv')
# y.to_csv('y.csv')

In [ ]:
#Validation function
n_folds = 5

def mae_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    mae= np.sqrt(-cross_val_score(model, train.values, y_train.values, scoring="neg_mean_absolute_error", cv = kf))
    return(mae)

In [ ]:
# Base models

In [ ]:
# LGBM :
model_lgb = lgb.LGBMRegressor(objective='regression',n_estimators=200)
mae = mae_cv(model_lgb)
print("lgb mae: \n", mae)
print("lgb score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [ ]:
model_lgb.fit(train, y)
y_pred = model_lgb.predict(test)
y_pred[:10]

In [ ]:
print(df_testing.shape)
print(y_pred.shape)

In [ ]:
# df_submit=df_testing
# df_submit[df_testing.columns[0]]=df_testing[df_testing.columns[0]]
# df_submit[df_testing.columns[1]]=y_pred
# df_submit.head()

In [ ]:
# # XGBoost :
# model_xgb = xgb.XGBRegressor(n_estimators=200)
# mae = mae_cv(model_xgb)
# print("Xgboost mae: \n", mae)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [ ]:
# # RF :
# model_rf=RandomForestRegressor(n_estimators=20)
# mae = mae_cv(model_rf)
# print("RandomForest mae: \n", mae)
# print("\nRandomForest score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))